In [49]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


## PuLP: Esta librería nos proporciona un lenguaje para modelar y resolver problemas de optimización utilizando programación lineal.

In [40]:
import pulp as pulp
import numpy as np
import pandas as pd
import random

El Dataset es del Departamento de Comercio del Gobierno de la India

In [43]:
export_df = pd.read_csv('2018-2010_export.csv')
export_df.head(3)

,HSCode,Commodity,value,country,year
0,2,MEAT AND EDIBLE MEAT OFFAL.,0.18,AFGHANISTAN TIS,2018
1,3,"FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...",0.00,AFGHANISTAN TIS,2018
2,4,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...,12.48,AFGHANISTAN TIS,2018


Se realizara solo para un tipo de producto y para la gestion 2018[1]

In [44]:
p2_df = pd.read_csv('export_2.csv')
p2_df.head(12)

,HSCode,Commodity,value,country,year
0,2,MEAT AND EDIBLE MEAT OFFAL.,0.18,AFGHANISTAN TIS,2018
1,2,MEAT AND EDIBLE MEAT OFFAL.,50.75,ALGERIA,2018
2,2,MEAT AND EDIBLE MEAT OFFAL.,27.31,ANGOLA,2018
3,2,MEAT AND EDIBLE MEAT OFFAL.,2.28,ARMENIA,2018
4,2,MEAT AND EDIBLE MEAT OFFAL.,0.51,AZERBAIJAN,2018
5,2,MEAT AND EDIBLE MEAT OFFAL.,NaN,BAHAMAS,2018
6,2,MEAT AND EDIBLE MEAT OFFAL.,24.68,BAHARAIN IS,2018
7,2,MEAT AND EDIBLE MEAT OFFAL.,9.06,BANGLADESH PR,2018
8,2,MEAT AND EDIBLE MEAT OFFAL.,0.08,BENIN,2018
9,2,MEAT AND EDIBLE MEAT OFFAL.,5.09,BHUTAN,2018


In [45]:

# Ejemplo del problema de transporte de las cervezas utilizando PuLP
# Creamos la variable prob que contiene los datos del problema
prob = pulp.LpProblem("Problema de exportacion de la India del producto 2", pulp.LpMinimize)

In [47]:
cervecerias = ["Producto_2"]

# diccionario con la capacidad de oferta de cada cerveceria
oferta = {"Producto_2": 1800}

# Creamos la lista de los bares o nodos de demanda
bares = ["AFGHANISTAN TIS",
         "ALGERIA",
         "ANGOLA",
         "ARMENIA",
         "AZERBAIJAN",
         "BAHAMAS",
         "BAHARAIN IS",
         "BANGLADESH PR",
         "BENIN",
         "BHUTAN",
         "BOTSWANA",
         "BRAZIL"]

# diccionario con la capacidad de demanda de cada bar
demanda = {"AFGHANISTAN TIS": 1,
             "ALGERIA": 50,
             "ANGOLA":27,
             "ARMENIA": 3,
             "AZERBAIJAN": 1,
             "BAHAMAS":0,
             "BAHARAIN IS":25,
             "BANGLADESH PR":10,
             "BENIN":1,
             "BHUTAN":6,
             "BOTSWANA":0,
             "BRAZIL":0}

# Lista con los costos de transporte de cada nodo
costos = [[50,40,50, 100, 500, 200, 800, 700,600, 550, 900, 10]]
# Convertimos los costos en un diccionario de PuLP
costos = pulp.makeDict([cervecerias, bares], costos,0)

# Creamos una lista de tuplas que contiene todas las posibles rutas de tranporte.
rutas = [(c,b) for c in cervecerias for b in bares]


In [48]:

# creamos diccionario x que contendrá la candidad enviada en las rutas
x = pulp.LpVariable.dicts("ruta", (cervecerias, bares), 
                        lowBound = 0,
                        cat = pulp.LpInteger)

# Agregamos la función objetivo al problema
prob += sum([x[c][b]*costos[c][b] for (c,b) in rutas]), \
            "Suma_de_costos_de_transporte"

# Agregamos la restricción de máxima oferta de cada cervecería al problema.
for c in cervecerias:
    prob += sum([x[c][b] for b in bares]) <= oferta[c], \
            "Suma_de_Productos_que_salen_de_cervecerias_%s"%c

# Agregamos la restricción de demanda mínima de cada bar
for b in bares:
    prob += sum([x[c][b] for c in cervecerias]) >= demanda[b], \
    "Sum_of_Products_into_Bar%s"%b
                   
# Los datos del problema son exportado a archivo .lp
prob.writeLP("problemaDeTransporte.lp")

# Resolviendo el problema.
prob.solve()

# Imprimimos el estado del problema.
print("Status: {}".format(pulp.LpStatus[prob.status]))

# Imprimimos cada variable con su solución óptima.
for v in prob.variables():
    print("{0:} = {1:}".format(v.name, v.varValue))

# Imprimimos el valor óptimo de la función objetivo   
print("Costo total de transporte = {}".format(prob.objective.value()))

Status: Optimal
ruta_Producto_2_AFGHANISTAN_TIS = 1.0
ruta_Producto_2_ALGERIA = 50.0
ruta_Producto_2_ANGOLA = 27.0
ruta_Producto_2_ARMENIA = 3.0
ruta_Producto_2_AZERBAIJAN = 1.0
ruta_Producto_2_BAHAMAS = 0.0
ruta_Producto_2_BAHARAIN_IS = 25.0
ruta_Producto_2_BANGLADESH_PR = 10.0
ruta_Producto_2_BENIN = 1.0
ruta_Producto_2_BHUTAN = 6.0
ruta_Producto_2_BOTSWANA = 0.0
ruta_Producto_2_BRAZIL = 0.0
Costo total de transporte = 35100.0


[1] URL:  https://www.kaggle.com/lakshyaag/india-trade-data